# Optimizing Mapper parameters

Mapper parameters are notoriously difficult to choose. Knowledge of your data can greatly narrow down the possibilities for your Mapper's filter function, covering scheme and/or clustering algorithm, but more fine-grained hyperparameters (e.g. ``eps`` in the case of ``DBSCAN``) can be difficult to guess.

In this notebook, we implement an idea described in Sect. 3.1 of [N. Chalapathi's BSc thesis](https://www.cs.utah.edu/docs/techreports/2021/PDF/UUCS-21-009.pdf), namely to use the *Akaike information criterion* to optimize Mapper hyperparameters.

Since ``giotto-tda`` implements Mapper as a ``scikit-learn`` pipeline, it inherits convenient frameworks for creating parameter grids and selecting the best combination.

In [ ]:
# Data wrangling
import numpy as np

# Data viz
from gtda.plotting import plot_point_cloud

# TDA magic
from gtda.mapper import (
    make_mapper_pipeline,
    Projection,
    OneDimensionalCover,
    MapperInteractivePlotter
    )

# ML tools
from joblib import Parallel, delayed
from sklearn import datasets
from sklearn.cluster import DBSCAN
from hdbscan import HDBSCAN
from sklearn.base import clone
from sklearn.model_selection import ParameterGrid

In [ ]:
X, _ = datasets.make_circles(n_samples=200, noise=0.05, factor=0.4, random_state=42)

plot_point_cloud(X)

In [ ]:
def compute_aic(X, nodes_elements, X_idx_to_node_idxs):
    """Custom algorithm for computing AIC-based Mapper graph score."""
    ## 0. Preparations
    #    - extract number of samples and ambient dimension
    N, d = X.shape

    ## 1. Compute the centroids of all Mapper nodes
    centroids = np.array([np.mean(X[node_elements], axis=0)
                          for node_elements in nodes_elements])

    ## 2. Produce a hard clustering by least distance to possible centroids
    #  2.1 Unique global cluster labels
    labels_unique = np.empty(N, dtype=np.int64)
    for i in range(N):
        possible_nodes = X_idx_to_node_idxs[i]
        if len(possible_nodes) == 1:
            labels_unique[i] = possible_nodes[0]
        else:
            rel_idx = np.argmin([np.linalg.norm(X[i] - centroids[node_id])
                                 for node_id in possible_nodes])
            labels_unique[i] = possible_nodes[rel_idx]
    #  2.2 List of clusters
    clusters = []
    for i, label in enumerate(labels_unique):
        n_clusters_now = len(clusters)
        to_add = label + 1 - n_clusters_now
        if to_add > 0:
            clusters += [[] for _ in range(to_add)]
        clusters[label].append(i)
    clusters = [np.array(c) for c in clusters if c]

    ## 3. Compute AIC for hard clustering
    k = len(clusters)
    cluster_sizes = np.array(list(map(len, clusters)))
    sigma_sq = np.sum((X - centroids[labels_unique])**2) / (d * (N - k))
    aic = 2 * np.sum(cluster_sizes * np.log(cluster_sizes))
    aic -= 2 * N * np.log(N) 
    aic -= N * d * np.log(2 * np.pi * sigma_sq) 
    aic -= d * (N - k)
    aic -= 2 * k * (d + 1)

    return aic

In [ ]:
def AIC_score(pipeline_params):
    """Compute the AIC score for a given Mapper pipeline on input data X."""

    ## 1. Replace current pipeline params with new ones in `pipeline_params`, and fit
    _pipeline = clone(pipeline)
    _pipeline.set_params(**pipeline_params)
    _pipeline.fit(X)

    ## 2. Extract information from Mapper graph:
    #    `node_elements`: elements of X in each node
    #    `X_idx_to_node_ids`: a mapping
    #        data index -> {global node IDs of all Mapper nodes the point belongs to}
    graph = _pipeline.named_steps["nerve"].graph_
    nodes = graph.vs
    
    # 2.1 `node_elements`
    nodes_elements = nodes["node_elements"]

    # 2.2 `X_idx_to_node_ids`
    global_ids = {(nodes[i]["pullback_set_label"], nodes[i]["partial_cluster_label"]): i
                  for i in range(len(nodes))}
    labels = _pipeline.named_steps["clustering"].labels_
    X_idx_to_node_idxs = [[global_ids[tup] for tup in labels[i]]
                          for i in range(len(X))]

    # Compute AIC score and return
    return pipeline_params, compute_aic(X, nodes_elements, X_idx_to_node_idxs)

In [ ]:
pipeline = make_mapper_pipeline(filter_func=Projection(columns=1),
                                cover=OneDimensionalCover())

In [ ]:
param_grid = {"clusterer": (HDBSCAN(), DBSCAN()),
              "cover__n_intervals": (3, 5, 10, 15),
              "cover__overlap_frac": (0.1, 0.2, 0.3),
              "cover__kind": ("uniform", "balanced")}

In [ ]:
# Run grid search in parallel using all available cores
grid_search_results = Parallel(n_jobs=-1)(
    delayed(AIC_score)(pipeline_params)
    for pipeline_params in ParameterGrid(param_grid)
    )

In [ ]:
best_idx = np.argmax([r[1] for r in grid_search_results])
best_grid, best_score = grid_search_results[best_idx]
best_grid

In [ ]:
pipeline.set_params(**best_grid)
plotter = MapperInteractivePlotter(pipeline, X)
plotter.plot()